# WEEK1
> ## **卷积运算——边缘检测实例:直线检测**    

左图对应一个半白半灰的图像(灰度值)，通过中间的filter(kernal,卷积核)进行**卷积运算(对应元素相乘后求和)**以后得到右边的图像，高亮/高暗部分就是一个垂直边缘，上图代表由明到暗，下图代表由暗到明。(存在一个较大的过渡部分——原因是图片较小，当图片维度扩大这种情况会被消除。)          
<img style="float: center;" src="course_4_pics/week_1_1.PNG" width=600 height=600> 

深度学习当中，将Filter中的九个参数利用反向传播进行学习。
<img style="float: center;" src="course_4_pics/week_1_2.PNG" width=100 height=100> 
> ##  **Padding**   

- 卷积运算带来的两个问题:
1. 输出缩小(多层卷积运算导致图像缩小)     
2. 图像边缘信息丢失(图像边缘进行的卷积运算次数小于图像中心的)         

Padding可以使**丢失信息以及角落与边缘的效果被削弱的程度变小。**                 

### Padding操作      
- 图像大小是$n\times n$      
- Filter大小$f\times f$       
- 经过一次卷积运算以后图像大小——$(n-f+1)\times (n-f+1)$。       
- Padding操作会在原图片的周围加上一圈像素,$if\ p = padding = 1$，图像大小变为$(n+2p-f+1)\times (n+2p-f+1)$。

    
### Padding方法  
1. **Valid Convolution**：NO Padding   
2. **Same Convocation**：Pad到输入与输出相同，即$p = \frac{f-1}{2}$      
($f$——Filter的维度通常是奇数，一是为了**pad的对称性**，二是奇数的Filter**易于指定一个中心点以指定这个Filter的位置**)         

> ## Strided——步幅(卷积步长)      

卷积步长指两个相邻的卷积操作之间移动的像素格数，在步幅$stride = s$的条件下卷积后得到的特征图大小：$\cfrac{n+2p-f}{s}\times \cfrac{n+2p-f}{s}$。        
如果前公式的商不是一个整数，通常使用向下取整($\lfloor \cfrac{n+2p-f}{s} \rfloor$),当卷积核有跳出图像范围的部分，那么选择不进行这次卷积操作。     
> ## 三维卷积     

### 立体卷积操作：      
处理的图像具有体积(RGB图像，有三个通道)，假设一个RGB图片，有RGB三层的亮度，那么这个图片即有三个维度,即```长(height)-宽(width)-通道个数(channels)```，通道个数需要与Filter相同。   

**卷积核也是一个立体** ，具有$f\times f \times 通道数$。 图片上的一个区域与Filter做一个卷积运算得到```f*f*channels```大小的立方体，**求和以后得到对应图片这个区域的值，** 最后铺在**二维矩阵**上。
例:如果需要对R通道进行边缘检测，那么在卷积核的第一层设置边缘检测的数字分布即可
<img style="float: center;" src="course_4_pics/week_1_3.PNG" width=600 height=600>       
   
### 多通道的卷积         
如果想要检测多个特征，那么可以使用多个卷积核同时对一张图进行操作。最后将不同卷积核得出的通道叠在一起。
<img style="float: center;" src="course_4_pics/week_1_4.PNG" width=600 height=600> 

> ## 卷积神经网络       
### 卷积层
- **结构**     
上一层的多通道图片的激活值$a^{[0]}$，通过卷积核$w^{[1]}$卷积操作生成$w^{[1]}w^{[0]}$，在经过激活函数生成该卷积核的通道$Relu(w^{[1]}w^{[0]}+b^{[1]})$，最后多个卷积核的通道叠加输入下一层。
<img style="float: center;" src="course_4_pics/week_1_5.PNG" width=600 height=600> 
- **参数**
$f^{[l]}=filter\ size\\   
p^{[l]}=padding\\     
s^{[l]}=stride\\
输出图片的深度n_c^{[l]} = 该层使用的filter个数,每个Filter输出一个通道\\
每个Filter的维度是f^{[l]}\times f^{[l]}\times n_c^{[l-1]}，Filter深度与上一层图片相同\\
Activation:a^{[l]}= n^{[l]}_H\times n^{[l]}_W\times n^{[l]}_c \\
当执行批量mini\ batch梯度下降，有m个例子时,则A^{[l]} = m \times  n^{[l]}_H\times n^{[l]}_W\times n^{[l]}_c\\  
Weights = f^{[l]}\times f^{[l]}\times n_c^{[l-1]}\times n_c^{[l]},n_c^{[l]}是l层的输出的通道数，也就是l层卷积核个数\\   
Bias:(1,1,1,n_c^{{l}}),一个卷积核内使用同一个Bias\\
Input:n^{[l-1]}_H\times n^{[l-1]}_W\times n^{[l-1]}_c \\ 
Output:n^{[l]}_H\times n^{[l]}_W\times n^{[l]}_c      \\
n^{[l]}_W = [\cfrac{n^{[l-1]}_W+2p^{[l]}-f{[l]}}{s^{[l]}}+1],n_H同理$

### 卷积神经网络基础结构
- Input输入层：输入图片
- Convolution卷积层：图片经过卷积层进行卷积操作，然后通过激活函数，最后进行Padding
- Pooling池化层：卷积层输出的多通道图片通过池化层进行池化
- ....多个卷积层与池化层，最终输出一个多通道图片
- Fully Connected全连接层：多通道图片进行展平，最终输入全连接层，外连接softmax/logistics
> ## Pooling       
### **Max Pooling**:       
用一个$n\times n$的筛选器去筛选该区域内的最大值(和卷积核采用一样的方法进行移动)。       
    当在某个区域可能**提取/识别到了某个特征，那么该区域内存在一个特别大的值**。如果某个区域内最大值仍然较小，那么可以认为该区域内没有提取到这样的值。
1. **Max Pooling内没有参数，因此Gradient Descent不改变该层的任何值。**      
2.最大池化层很少增加padding
<img style="float: center;" src="course_4_pics/week_1_7.PNG" width=600 height=600> 


### **average pooling**:
筛选区域内的平均值       

### 超参数    
- $f$，池化筛选器的尺寸
- $s$，筛选器移动的步幅    
- Max or Avg

> ## 为什么使用卷积神经网络    
### 参数量远小于全连接
-  **稀疏连接**   
卷积后的图像的一个像素仅仅与原图像上一个filter大小的区域进行连接(稀疏连接)，图上的其他像素对该区域的检测不起作用。(图片卷积前后颜色对应的部分)     
<img style="float: center;" src="course_4_pics/week_1_8.PNG" width=600 height=600> 
-  **共享参数**    
多个像素区域共享一个filter，代表一个filter能在图上的不同位置进行检测，因此就减少参数数量。(假设给每一层的像素与下一层的像素进行全连接，参数数量非常大)。之所以可以使用共享参数，是因为在图片中一个区域能够进行特征检测的卷积核，在图片的其他部分也很有可能起到作用。     
     
